### 이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

로컬에서 돌리셔도 되지만, colab에서 GPU로 돌려보는 것을 권장합니다!

## 데이터 로드 및 탐색

In [ ]:
%%capture
!pip install transformers

In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('./drive/MyDrive/KUBIG25/WEEK4/bbc-text.csv') # bbc-text.csv 파일 경로

In [4]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [5]:
print(len(df))

2225


In [6]:
df.groupby('category').count()

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Dataset

In [8]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):                          #레이블 반환
        return self.labels

    def __len__(self):                          #데이터 샘플 수 반환
        return len(self.labels)

    def get_batch_labels(self, idx):            #특정 인덱스의 레이블 반환
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):             #특정 인덱스의 텍스트 반환
        return self.texts[idx]

    def __getitem__(self, idx):                 #특정 인덱스의 레이블/텍스트 반환

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [9]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()                      #Bert기반 텍스트 분류 모델, 드롭아웃 확률: 0.5

        self.bert = BertModel.from_pretrained('bert-base-cased')    #사전학습 모델 불러오기
        self.dropout = nn.Dropout(dropout)                          #드롭아웃 레이어 구성
        self.linear = nn.Linear(768, 5)                             #단순 선형 레이어
        self.relu = nn.ReLU()                                       #활성화(Relu)

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)    #pooled_output -> CLS 토큰 임베딩
        dropout_output = self.dropout(pooled_output)                                                #dropout 적용
        linear_output = self.linear(dropout_output)                                                 #다섯가지 클래스에 대한 예측 출력
        final_layer = self.relu(linear_output)                                                      #활성화 함수 적

        return final_layer

In [10]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)                                 #train data와 test data에 Dataset 메서드 적용

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)   #Dataloader 생성, 배치크기는 2
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()                                                   #손실 함수 정의 -> CrossEntropyLoss
    optimizer = Adam(model.parameters(), lr= learning_rate)                             #옵티마이저 -> Adam

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):                                                     #epochs만큼 학습 진행

            total_acc_train = 0                                                         #accuracy와 loss 초기화
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):                     #배치 단위로 학습

                train_label = train_label.to(device)                                    #레이블 이동(CPU/GPU)
                mask = train_input['attention_mask'].to(device)                         #attention mask 및 token id 이동
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)                                          #forward 진행

                batch_loss = criterion(output, train_label.long())                      #손실 값 계산
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()                #정확도 계산
                total_acc_train += acc

                model.zero_grad()                                                       #그래디언트 초기화
                batch_loss.backward()                                                   #역전파 진행
                optimizer.step()                                                        #옵티마이저로 가중치 업데이트

            total_acc_val = 0                                                           #정확도 초기화(val 데이터)
            total_loss_val = 0                                                          #손실 초기화(val 데이터)

            with torch.no_grad():                                                       #검증 단계 -> 그래디언트 업데이트X

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)                                    #레이블 이동
                    mask = val_input['attention_mask'].to(device)                       #attention mask 이동
                    input_id = val_input['input_ids'].squeeze(1).to(device)             #id 이동

                    output = model(input_id, mask)                                      #예측 값 생성

                    batch_loss = criterion(output, val_label.long())                    #손실 값 계산
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()              #정확도 계산
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')


In [11]:
def evaluate(model, test_data):

    test = Dataset(test_data)                                                           #test데이터에 Dataset 메서드 적용

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)                   #Dataloader 구현, 배치 크기 2

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0                                                                  #정확도 0으로 초기화
    with torch.no_grad():                                                               #평가 단계에서는 그래디언트 계산X

        for test_input, test_label in test_dataloader:                                  #역시 동일하게 배치단위로 활용

              test_label = test_label.to(device)                                        #레이블 이동
              mask = test_input['attention_mask'].to(device)                            #attention mask 이동
              input_id = test_input['input_ids'].squeeze(1).to(device)                  #id 이동

              output = model(input_id, mask)                                            #모델 예측 수행

              acc = (output.argmax(dim=1) == test_label).sum().item()                   #정확도 계
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [12]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [13]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

100%|██████████| 890/890 [03:02<00:00,  4.86it/s]


Epochs: 1 | Train Loss:  0.760 | Train Accuracy:  0.331 | Val Loss:  0.531 | Val Accuracy:  0.775


100%|██████████| 890/890 [03:03<00:00,  4.84it/s]


Epochs: 2 | Train Loss:  0.296 | Train Accuracy:  0.920 | Val Loss:  0.148 | Val Accuracy:  0.973


In [14]:
evaluate(model, df_test)

Test Accuracy:  0.982


optional) 다양한 시도를 해보셨다면 시도 별 간단한 해석도 달아주세요! 🤗